In [30]:
%run model_functions.ipynb

In [4]:
genre = 'newspapers'
all_words = pd.read_csv('data/train/{}/words.csv'.format(genre), na_filter=False)
display(len(all_words))
all_words.head()

35740

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers


In [5]:
all_words['text_id'].nunique(), all_words['sentence_id'].nunique()

(49, 1788)

We have 10 texts and 2023 sentences. Let's try to find 2 unique topics per text!

Let's also only consider content words, so we don't consider words like "the", "and", "be".

In [6]:
content_words = build_content_words(all_words)
display(len(content_words))
content_words.head()

18527

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers


In [7]:
def get_topic_assignments(components, vocab):
    n_top_words = 10
    topic_words = {}
    
    for topic, component in enumerate(components):
        # Argsort and reverse iterate to get vocab indices of top words for each topic. 
        top_word_ids = np.argsort(component)[::-1][:n_top_words]

        # Store the words most relevant to the topic
        topic_words[topic] = [vocab.iloc[i].name for i in top_word_ids]
        
    for topic, words in topic_words.items():
        print('Topic: %d' % topic)
        print('  %s' % ', '.join(words))

In [8]:
# Turn lemmas of document to a document word matrix.
def document_word_matrix(words, vocabulary):
    indices = []
    data = []
    indptr = [0]
    for i, word in words['lemma'].iteritems():
        # Used to generate a row of 0's and 1's for sparse matrix
        unique_unigram_indices = set()
        # ID for an unknown word is == len(vocabulary).
        if word not in vocabulary.index:
            unique_unigram_indices.add(len(vocabulary))
        else:
            unique_unigram_indices.add(vocabulary.index.get_loc(word))

        # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
        indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
        data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
        indptr.append(indptr[-1]+len(unique_unigram_indices))

        if not i % 2000:
            print('...', end='', flush=True)

    return csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(vocabulary)+1))

Assumption: lemmatized content words -> better topics.

In [9]:
len(content_words)

18527

In [10]:
sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')[['sentence_id', 'paragraph_id']]
dataset = (
    content_words
     .merge(sentences)
     .merge(pd.Series(build_paragraphs(content_words).index))
)
display(len(dataset))
dataset.head()  # The words that belong to paragraphs.

17969

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre,paragraph_id
0,26,SEEMS,seem,VVZ,,,2,a1e-fragment01,a1e,newspapers,3
1,28,Roland,roland,NP0,,,2,a1e-fragment01,a1e,newspapers,3
2,29,Franklin,franklin,NP0,,,2,a1e-fragment01,a1e,newspapers,3
3,31,latest,late,AJS,,,2,a1e-fragment01,a1e,newspapers,3
4,32,unbundler,unbundler,NN1,,,2,a1e-fragment01,a1e,newspapers,3


In [11]:
def model_all_topics(dataset, n_topics=20, max_iter=2000, n_jobs=-1):
    vocab = dataset[['lemma']].drop_duplicates().set_index('lemma')

#     doc_topic_dists = []
    print(len(dataset), len(vocab))
    X = document_word_matrix(dataset, vocab)
    print(X.shape, len(dataset), len(vocab))

    lda = LatentDirichletAllocation(
        n_components=n_topics, max_iter=max_iter, random_state=0,
        verbose=10, n_jobs=n_jobs
    )

#     doc_topic_dist = lda.fit_transform(X)
    lda.fit(X)
    
    return lda
    

The next cell fits and saves an LDA model to models/ldamodel.joblib

In [12]:
lda_model = model_all_topics(dataset, 20, 2000, -1)

dump(lda_model, 'models/lda_{}.joblib'.format(genre))

17969 5811
...........................(17969, 5812) 17969 5811


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.9s remaining:    3.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 2000


...

iteration: 1999 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2000 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


['models/lda_newspapers.joblib']

In [14]:
lda_model = load('models/lda_{}.joblib'.format(genre))

In [15]:
lda_model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=2000, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=10)

In [16]:
vocab = dataset[['lemma']].drop_duplicates().set_index('lemma')
print(len(vocab))
vocab.head(10)

5811


""
lemma
seem
roland
franklin
late
unbundler
appear
uk
make
fatal


In [17]:
# n_topics
def calc_t_given_d(doc_topic_dist):
    total_d = sum((sum(dtd) for dtd in doc_topic_dist))
    t_given_d = [
        sum(tdd)/total_d for tdd in zip(*doc_topic_dist)
    ]
    return t_given_d

# n_topics X n_vocabulary
def calc_w_given_t(components):
    return components / components.sum(axis=1)[:, np.newaxis]

# n_vocabulary
def calc_w(components):
    return components.sum(axis=0)/sum(sum(arr) for arr in components)

# n_topics
def calc_t(doc_topic_dist):
    return doc_topic_dist.sum(axis=0)/sum(sum(arr) for arr in doc_topic_dist)

# n_vocabulary X n_topics
def calc_t_given_w(components, doc_topic_dist):
    w_given_t = calc_w_given_t(components)
    p_w = calc_w(components)
    p_t = calc_t(doc_topic_dist)

    t_given_w = list(zip(*((w_given_t / p_w) * p_t[:,None])))
    return t_given_w

In [18]:
gt_feature = []
for paragraph, group in dataset.groupby('paragraph_id'):
    sparse_matrix = document_word_matrix(group, vocab)
    paragraph_topic_dist = lda_model.transform(sparse_matrix)
    
    t_given_w = calc_t_given_w(lda_model.components_, paragraph_topic_dist)
    t_given_d = calc_t_given_d(paragraph_topic_dist)
    for word in group.itertuples():
        cos_sim = cosine(
            t_given_w[vocab.index.get_loc(
                word.lemma)],
            t_given_d
        )
#         dataset.at[dataset['word_id']==word.word_id, 'GT'] = cos_sim
        gt_feature.append([word.word_id, cos_sim])

    print('Calculating for paragraph {}'.format(paragraph))

...Calculating for paragraph 3
Calculating for paragraph 4
Calculating for paragraph 5


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 6


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 7
Calculating for paragraph 8
Calculating for paragraph 9


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 1038
Calculating for paragraph 1039
Calculating for paragraph 1040


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 1041
Calculating for paragraph 1042


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


In [24]:
gt_feature = pd.DataFrame(gt_feature, columns=['word_id', 'GT'])
display(len(gt_feature))
gt_feature.head()

17969

,word_id,GT
0,26,0.542670
1,28,0.575656
2,29,0.741269
3,31,0.791323
4,32,0.494945


In [26]:
temp = dataset.merge(gt_feature, left_on='word_id', right_on='word_id')
display(len(temp))
temp.head()

17969

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre,paragraph_id,GT_x,GT_y
0,26,SEEMS,seem,VVZ,,,2,a1e-fragment01,a1e,newspapers,3,NaN,0.542670
1,28,Roland,roland,NP0,,,2,a1e-fragment01,a1e,newspapers,3,NaN,0.575656
2,29,Franklin,franklin,NP0,,,2,a1e-fragment01,a1e,newspapers,3,NaN,0.741269
3,31,latest,late,AJS,,,2,a1e-fragment01,a1e,newspapers,3,NaN,0.791323
4,32,unbundler,unbundler,NN1,,,2,a1e-fragment01,a1e,newspapers,3,NaN,0.494945


In [21]:
gt_feature.to_csv('features/GT_{}.csv'.format(genre))

-----------

In [31]:
genre = 'newspapers'

with open('results/GT_{}.txt'.format(genre), 'w') as f:
    train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_train, test_fold_ids, vocabulary, y_train = preprocess_data(train_set, want_gt=True)

    unigram_classifier = UnigramClassifier(
        verbose=10, random_state=0, C=10, penalty='l1',
        solver='liblinear', class_weight='balanced', max_iter=500,
        vocabulary=vocabulary
    )

    C = np.logspace(-2,4,20)
    penalty = ['l1', 'l2']
    hyperparameters = {'C':C, 'penalty':penalty}
    clf = GridSearchCV(unigram_classifier, hyperparameters,
                       cv=PredefinedSplit(test_fold_ids),
                       verbose=10, scoring='f1', n_jobs=-1)

    clf.fit(X_train, y_train)

    test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_test, _, _, y_test = preprocess_data(test_set, want_gt=True)
    preds = clf.predict(X_test)

    result = classification_report(y_test, preds, output_dict=True)
    print(result)
    f.write(json.dumps(result))

------(18527, 7691) 7690
...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    7.1s remaining:    7.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    8.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 3
Calculating for paragraph 4


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 5
Calculating for paragraph 6
Calculating for paragraph 7

...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 1040
Calculating for paragraph 1041


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


Calculating for paragraph 1042


TypeError: expected dimension <= 2 array or matrix

In [33]:
X.shape

NameError: name 'X' is not defined

In [34]:
len(gt_feature)

17969